In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
# spark = SparkSession.builder \
#     .master("spark://Yokis-MacBook-Pro.local:7077") \
#     .appName('test') \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName('test') \
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.13:0.39.1') \
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true') \
    .config('spark.sql.catalogImplementation', 'hive') \
    .config('spark.executor.extraClassPath', '/opt/homebrew/opt/scala@2.13/libexec/lib') \
    .config('spark.driver.extraClassPath', '/opt/homebrew/opt/scala@2.13/libexec/lib') \
    .getOrCreate()

print("Spark Version:", spark.version)
print("Scala Version:", spark.sparkContext.getConf().get("spark.scala.version"))

Spark Version: 3.5.1


NameError: name 'scala' is not defined

In [3]:
spark

In [5]:
from pyspark import SparkConf
SparkConf().getAll()

[('spark.app.name', 'test'),
 ('spark.app.submitTime', '1720028102161'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'spark://Yokis-MacBook-Pro.local:7077')]

In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [5]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [6]:
# combine dataframes into one 
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [7]:
common_columns = []
yellow_columns = set(df_yellow.columns)
for col in df_green.columns: 
    if col in yellow_columns: 
        common_columns.append(col)

In [8]:
from pyspark.sql import functions as F

In [9]:
df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [10]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [11]:
df_trips_data.createOrReplaceTempView('trips_data')

In [12]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [13]:
df_result.write.parquet('data/report/revenue/', mode="overwrite")